# Deploy Web App on Azure Container Services (AKS)


In [35]:
resource_group = "fbaksrg" # Feel free to modify these
location = "eastus"

image_name = "fboylu/tfkeras" 
selected_subscription = "'Team Danielle Internal'"
cluster_name = "fbAKSCluster"

In [2]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FD9TULRUB to authenticate.
CloudName    Name                           State    TenantId                              IsDefault
-----------  -----------------------------  -------  ------------------------------------  -----------
AzureCloud   Boston DS Dev                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Internal Consumption           Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Team Danielle Internal         Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47  True
AzureCloud   Boston Engineering             Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   ADLTrainingMS                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Ads Eng Big Data Subscription  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Data Wrangling Preview         Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Data Wrangling devel

In [4]:
!az account set --subscription $selected_subscription

In [3]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [6]:
!az provider show -n Microsoft.ContainerService

{
  "additionalProperties": {
    "authorization": {
      "applicationId": "7319c514-987d-4e9b-ac3d-d38c4f427f4c",
      "managedByRoleDefinitionId": "8e3af657-a8ff-443c-a75c-2fe8c4bcb635",
      "roleDefinitionId": "1b4a0c7f-2217-416f-acfa-cf73452fdc1c"
    }
  },
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/providers/Microsoft.ContainerService",
  "namespace": "Microsoft.ContainerService",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "additionalProperties": {
        "capabilities": "None"
      },
      "aliases": null,
      "apiVersions": [
        "2017-07-01",
        "2017-01-31",
        "2016-09-30",
        "2016-03-30"
      ],
      "locations": [
        "Japan East",
        "Central US",
        "East US 2",
        "Japan West",
        "East Asia",
        "South Central US",
        "Australia East",
        "Australia Southeast",
        "Brazil South",
        "Southeast Asia",
        "West US",
        "North Central US",

In [10]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


In [9]:
%%time
!az aks create --resource-group $resource_group --name $cluster_name --node-count 1 --generate-ssh-keys -s Standard_NC6

SSH key files '/home/fboylu/.ssh/id_rsa' and '/home/fboylu/.ssh/id_rsa.pub' have been generated under ~/.ssh to allow SSH access to the VM. If using machines without permanent storage like Azure Cloud Shell without an attached file share, back up your keys to a safe location
Waiting for AAD role to propagate[################################    ]  90.0000%%Could not create a service principal with the right permissions. Are you an Owner on this project?
^CK - Running ...
Long-running operation wait cancelled.  


In [12]:
!sudo az aks install-cli

In [15]:
!az aks get-credentials --resource-group $resource_group --name $cluster_name

Merged "fbAKSCluster" as current context in /home/fboylu/.kube/config


In [16]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     20h       v1.7.7


In [48]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-2574232661-9rnfw               2/2       Running   0          21h
kube-system   kube-dns-v20-2253765213-nc3lc           3/3       Running   0          21h
kube-system   kube-dns-v20-2253765213-tlrxc           3/3       Running   0          21h
kube-system   kube-proxy-g4gs2                        1/1       Running   0          21h
kube-system   kube-svc-redirect-57f3x                 1/1       Running   0          21h
kube-system   kubernetes-dashboard-2898242510-5f9m0   1/1       Running   0          21h
kube-system   tunnelfront-2759728499-dgvc5            1/1       Running   0          21h


In [89]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/tfkeras",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":88,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "name": "bin",
                              "mountPath":"/usr/local/nvidia/bin" 
                          },
                          {
                              "name": "lib",
                              "mountPath":"/usr/local/nvidia/lib64" 
                          },
                          {
                              "name": "libcuda",
                              "mountPath":"/usr/lib/x86_64-linux-gnu/libcuda.so.1" 
                          },
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "bin",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384/bin"
                      },
                  },
                  {
                      "name": "lib",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384"
                      },
                  },
                  {
                      "name": "libcuda",
                      "hostPath":{
                          "path":"/usr/lib/x86_64-linux-gnu/libcuda.so.1"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":88
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [90]:
import json

In [91]:
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4,sort_keys=True)
        outfile.write('\n\n')

In [92]:
write_json_to_file(app_template, 'az-dl.json')

In [93]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

In [94]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/tfkeras",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 88,
                                "name": "model"
                            }
                        ],
                   

In [96]:
!kubectl create -f az-dl.json

deployment "azure-dl" created
service "azure-dl" created


In [100]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                             SOURCE                              MESSAGE
8s          23h          1383      aks-nodepool1-28016997-0.151e0b00f2c2fa81    Node                                     Warning   FailedNodeAllocatableEnforcement   kubelet, aks-nodepool1-28016997-0   Failed to update Node Allocatable Limits "": failed to set supported cgroup subsystems for cgroup : Failed to set config for supported subsystems : failed to write 59076296704 to memory.limit_in_bytes: write /var/lib/docker/overlay2/b13d017f80b0afbea6242f387627df263ea463c385c0655defa6221c49714e66/merged/sys/fs/cgroup/memory/memory.limit_in_bytes: invalid argument
2m          2m           1         azure-dl-1710293173-p2ggn.151e564d80b477ec   Pod          spec.containers{azure-dl}   Normal    Killing                            kubelet, aks-nodepool1-28016997-0   Killing container wi

In [99]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-3344564842-0kbg0               1/1       Running   0          1m
kube-system   heapster-2574232661-9rnfw               2/2       Running   0          22h
kube-system   kube-dns-v20-2253765213-nc3lc           3/3       Running   0          22h
kube-system   kube-dns-v20-2253765213-tlrxc           3/3       Running   0          22h
kube-system   kube-proxy-g4gs2                        1/1       Running   0          22h
kube-system   kube-svc-redirect-57f3x                 1/1       Running   0          22h
kube-system   kubernetes-dashboard-2898242510-5f9m0   1/1       Running   0          22h
kube-system   tunnelfront-2759728499-dgvc5            1/1       Running   0          22h


In [105]:
!kubectl logs azure-dl-3344564842-0kbg0

2018-03-22 19:57:27,159 CRIT Supervisor running as root (no user in config file)
2018-03-22 19:57:27,161 INFO supervisord started with pid 7
2018-03-22 19:57:28,164 INFO spawned: 'program_exit' with pid 17
2018-03-22 19:57:28,165 INFO spawned: 'nginx' with pid 18
2018-03-22 19:57:28,166 INFO spawned: 'gunicorn' with pid 19
2018-03-22 19:57:29,198 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-03-22 19:57:29.558220: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-03-22 19:57:33,569 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-03-22 19:57:38.178449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: d333:00:00.0
totalMemory: 11.17GiB freeMe

In [102]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           3m


In [108]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.21.232   13.90.129.123   88:31864/TCP   59m


# Tear it all down

In [95]:
!kubectl delete -f az-dl.json

deployment "azure-dl" deleted
service "azure-dl" deleted


In [ ]:
!az aks delete -n $cluster_name -g $resource_group -y

In [ ]:
!az group delete --name $resource_group -y